# DataFrame Merging Benchmarks

Compare `catabra_pandas.merge_intervals` to [`janitor.conditional_join`](https://github.com/pyjanitor-devs/pyjanitor/tree/dev) and [`polars.join_where`](https://github.com/pola-rs/polars) in terms of time and memory efficiency.

The Appendix briefly discusses [`pandas.IntervalIndex`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.IntervalIndex.html#pandas.IntervalIndex).

In [ ]:
import numpy as np
import pandas as pd
import janitor
import polars as pl
import catabra_pandas

In [2]:
import memory_profiler, time

In [5]:
# our own profiling function, to measure time and memory simultaneously

def profile(func, args, kwargs, reps: int = 1, memory: bool = True, print_output: bool = True, size_fun=len):
    def _f():
        tic = time.time()
        for _ in range(reps - 1):
            func(*args, **kwargs)
        out = func(*args, **kwargs)
        return (out, (time.time() - tic) / reps)
    
    if memory:
        baseline = memory_profiler.memory_usage()[0]
        max_mem, (retval, t) = memory_profiler.memory_usage((_f, [], {}), retval=True, max_usage=True)
        incr = max_mem - baseline
    else:
        retval, t = _f()
        max_mem = incr = 0

    if print_output:
        print("output size:", size_fun(retval))
        if t >= 1.0:
            ts = "{:.2f} s".format(t)
        elif t >= 1e-3:
            ts = "{:.2f} ms".format(t * 1e3)
        else:
            ts = "{:.2f} us".format(t * 1e6)
        print("wall time:  ", ts)

        if memory:
            print("peak memory:", "{:.2f} MiB".format(max_mem))
            print("increment:  ", "{:.2f} MiB".format(incr))

    return retval, t, max_mem, incr

## Test 1: Equality and Interval-Containment Constraints

In [6]:
def create_random_frames(n_groups: int, group_size: int, seed: int = 42) -> tuple[pd.DataFrame, pd.DataFrame]:
    rng = np.random.RandomState(seed)

    left = pd.DataFrame(
        data=dict(
            group=rng.randint(0, n_groups, size=n_groups*group_size*10),
            start=rng.uniform(-10, 10, size=n_groups*group_size*10)
        )
    )
    left["stop"] = left["start"] + rng.uniform(1, 10, size=len(left))

    right = pd.DataFrame(
        data=dict(
            group=rng.randint(0, n_groups, size=n_groups*group_size),
            start=rng.uniform(15, 25, size=n_groups*group_size)
        )
    )
    right["stop"] = right["start"] + rng.uniform(1, 5, size=len(right))

    return left, right

In [8]:
left, right = create_random_frames(10_000, 100)
print(len(left))
print(len(right))

10000000
1000000


We want to merge `left` and `right` on column `"group"` (equality constraint), and additionally restrict the result to rows where `left["start"] <= right["stop"] <= left["stop"]`.

Let's start with `catabra_pandas.merge_intervals`:

In [9]:
out, wall, peak, incr = profile(
    catabra_pandas.merge_intervals,
    [left, right],
    dict(on="group", how="inner", left_start="start", left_stop="stop", right_start="stop", right_stop="stop", keep_order=False, copy=False)
)

output size: 1487230
wall time:   5.47 s
peak memory: 3108.66 MiB
increment:   2756.04 MiB


Compare that to `janitor.conditional_join`:

In [10]:
out_janitor, wall_janitor, peak_janitor, incr_janitor = profile(
    janitor.conditional_join,
    [left, right, ("group", "group", "=="), ("start", "stop", "<="), ("stop", "stop", ">=")],
    dict(how="inner")
)

output size: 1487230
wall time:   31.93 s
peak memory: 38761.12 MiB
increment:   38350.74 MiB


**janitor is 5 times slower and requires more than 10 times more intermediate memory!**

OK, what about `pl.join_where`?

In [13]:
left_pl = pl.from_pandas(left)
right_pl = pl.from_pandas(right)

In [12]:
out_pl, wall_pl, peak_pl, incr_pl = profile(
    left_pl.join_where,
    [right_pl, pl.col("group") == pl.col("group_right"), pl.col("start") <= pl.col("stop_right"), pl.col("stop") >= pl.col("stop_right")],
    {}
)

output size: 1487230
wall time:   9.91 s
peak memory: 50504.18 MiB
increment:   49866.89 MiB


**Polars is 2 times slower and requires almost 20 times more intermediate memory!**

## Test 2: Equality and Interval-Overlap Constraints

We use the same example as before, but this time match all rows from `left` and `right` with the same `"group"` and overlapping intervals (defined by `"start"` and `"stop"`).

`catabra_pandas.merge_intervals`:

In [10]:
out, wall, peak, incr = profile(
    catabra_pandas.merge_intervals,
    [left, right],
    dict(on="group", how="inner", left_start="start", left_stop="stop", right_start="start", right_stop="stop", keep_order=False, copy=False)
)

output size: 11616148
wall time:   9.73 s
peak memory: 3093.24 MiB
increment:   2682.39 MiB


`janitor.conditional_join`:

In [ ]:
out_janitor, wall_janitor, peak_janitor, incr_janitor = profile(
    janitor.conditional_join,
    [left, right, ("group", "group", "=="), ("start", "stop", "<="), ("stop", "start", ">=")],
    dict(how="inner")
)

output size: 11616148
wall time:   28.04 s
peak memory: 39164.60 MiB
increment:   38367.25 MiB


**janitor is still 3 times slower and requires more than 10 times more intermediate memory!**

`pl.join_where`:

In [14]:
out_pl, wall_pl, peak_pl, incr_pl = profile(
    left_pl.join_where,
    [right_pl, pl.col("group") == pl.col("group_right"), pl.col("start") <= pl.col("stop_right"), pl.col("stop") >= pl.col("start_right")],
    {}
)

output size: 11616148
wall time:   9.34 s
peak memory: 50786.97 MiB
increment:   49362.09 MiB


**Polars is on-par in terms of computation time, but still requires almost 20 times more intermediate memory!**

## Test 3: Single Inequality Constraint

This example is taken from [Polars' test suite](https://github.com/pola-rs/polars/blob/main/py-polars/tests/benchmark/test_join_where.py). The goal is to join two DataFrames based on a single inequality constraint and no equality constraints.

This use-case is covered by `catabra_pandas.merge_intervals`, since interval endpoints can be $\pm\infty$ - either explicitly, if the data type supports infinity, or implicitly by simply omitting the respective interval endpoint from the function call.

In [7]:
def east_west(n_rows_left: int = 5_000_000, n_rows_right: int = 5_000_000, seed: int = 42):
    # taken from https://github.com/pola-rs/polars/blob/main/py-polars/tests/benchmark/test_join_where.py
    
    rng = np.random.default_rng(seed)

    # Generate two separate datasets where revenue/cost are linearly related to
    # duration/time, but add some noise to the west table so that there are some
    # rows where the cost for the same or greater time will be less than the east table.
    east_dur = rng.integers(1_000, 10_000_000, n_rows_left)
    east_rev = (east_dur * 0.123).astype(np.int32)
    west_time = rng.integers(1_000, 500_000, n_rows_right)
    west_cost = west_time * 0.123
    west_cost += rng.normal(0.0, 1.0, n_rows_right)
    west_cost = west_cost.astype(np.int32)

    east = pd.DataFrame(
        {
            "id": np.arange(0, n_rows_left),
            "dur": east_dur,
            "rev": east_rev,
            "cores": rng.integers(1, 10, n_rows_left),
        }
    )
    west = pd.DataFrame(
        {
            "t_id": np.arange(0, n_rows_right),
            "time": west_time,
            "cost": west_cost,
            "cores": rng.integers(1, 10, n_rows_right),
        }
    )
    return east, west

In [15]:
east, west = east_west(50_000, 5000)
print(len(east))
print(len(west))

50000
5000


`catabra_pandas.merge_intervals`:

In [16]:
out, wall, peak, incr = profile(
    catabra_pandas.merge_intervals,
    [east, west],
    dict(how="inner", left_start="dur", left_stop="dur", right_stop="time", include_right_stop=False, keep_order=False, copy=False)
)

output size: 6381653
wall time:   167.26 ms
peak memory: 46827.27 MiB
increment:   670.46 MiB


`janitor.conditional_join`:

In [17]:
out_janitor, wall_janitor, peak_janitor, incr_janitor = profile(
    janitor.conditional_join,
    [east, west, ("dur", "time", "<")],
    {}
)

output size: 6381653
wall time:   84.79 ms
peak memory: 46686.77 MiB
increment:   682.80 MiB


**janitor is very fast, with comparable memory increment.**

`pl.join_where`:

In [18]:
east_pl = pl.from_pandas(east)
west_pl = pl.from_pandas(west)

In [26]:
out_pl, wall_pl, peak_pl, incr_pl = profile(
    east_pl.join_where,
    [west_pl, pl.col("dur") < pl.col("time")],
    {}
)

output size: 6381653
wall time:   34.56 ms
peak memory: 2449.11 MiB
increment:   157.16 MiB


**Polars is super fast, with negligible memory increment.**

## Conclusion

`catabra_pandas.merge_intervals` is a fast alternative to both `janitor.conditional_join` and `polars.join_where`. Our experiments reveal that the time- and memory gains can be significant, especially if the **involved DataFrames are big (>1M rows)** and **both equality and inequality constraints are present**.

The main downside of `catabra_pandas.merge_intervals` is that it can only handle interval overlap and -containment, and therefore covers only a subset of all possible inequality constraints. In real-world applications, this is often sufficient, though. Furthermore, every combination of inequalities can be represented as an equivalent combination of interval overlaps.

# Appendix: A Few Words on `pd.IntervalIndex`

* `IntervalIndex` is useful for working with interval data. If `df` has an `IntervalIndex`, then `df.loc[scalar]` returns
    all rows whose index contains `scalar`.
* `IntervalIndex` can be combined with other indexes in a `MultiIndex`. `df.loc[(i1, i2)]` selects rows based on both
    indexes. **BUT SEE CAVEATS BELOW!**
* `IntervalIndex` may contain overlapping intervals. In that case, `df.loc[scalar]` may return a `DataFrame` with multiple
    rows (just as when used with other, non-unique indexes).
* `IntervalIndex` allows `.loc`-indexing with lists of indices, e.g., `df.loc[[scalar1, scalar2, scalar3]]`. All matching
    rows are returned, as expected.
* Joining on `IntervalIndex` does *not* work as expected, neither `IntervalIndex` on `IntervalIndex`, nor `IntervalIndex` on
    scalar index. If `IntervalIndex` is joined on `IntervalIndex`, only identical intervals are joined.
* `IntervalIndex.overlaps` is only implemented for single intervals, not for other `IntervalIndex`.
* It seems that `IntervalIndex`-`MultiIndex` combos [have not been thorougly integrated into pandas yet](https://github.com/pandas-dev/pandas/issues/25298), but only work
"occasionally".

**WARNING!** Combining `IntervalIndex`, `MultiIndex` and lists of indices does *not* give the expected result, namely only *one*
row for each passed index. Example:

In [27]:
s = pd.Series(
    index=pd.MultiIndex.from_product(
        [pd.IntervalIndex.from_arrays([0, 1], [3, 6], closed='both'), pd.RangeIndex(2)]
    ),
    data=np.arange(4)
)
s

[0, 3]  0    0
        1    1
[1, 6]  0    2
        1    3
dtype: int32

In [ ]:
s.loc[(2.5, 1)]    # expected

[0, 3]  1    1
[1, 6]  1    3
dtype: int32

In [ ]:
s.loc[[(2.5, 1)]]  # unexpected

[0, 3]  1    1
dtype: int32

**Note**: The above behavior is independent of the order of `IntervalIndex` and `RangeIndex` within the `MultiIndex`.